In [1]:
%matplotlib inline

from getpass import getpass
from pandas_datareader import data
from datetime import datetime, timedelta
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import requests as r
import pandas as pd
import xml.etree.cElementTree as etree

init_notebook_mode(connected=True)

ws = 'http://ws.eoddata.com/data.asmx'
ns='http://ws.eoddata.com/Data'
exchange = 'ASX'
min_mcap = 100000000
min_days_traded = 250
start = '1999-01-01'
end = str(datetime.now())[:10]   

In [2]:
eoddata_username = getpass()

········


In [3]:
eoddata_password = getpass()

········


In [8]:
call = 'Login'
url = '/'.join((ws, call))
params = {'Username': eoddata_username, 'Password': eoddata_password}

s = r.Session()
response = s.get(url, params=params, stream=True)

if response.status_code == 200:
    xml_doc_root = etree.parse(response.raw).getroot()
    token = xml_doc_root.get('Token')
    
call = None
url = None
params = None
response = None
xml_doc_root = None

In [9]:
call = 'FundamentalList'
url = '/'.join((ws, call))
params = {'Token': token, 'Exchange': exchange}

response = s.get(url, params=params, stream=True)
xml_doc_root = etree.parse(response.raw).getroot()

In [10]:
symbol_list = []
for fundamental in xml_doc_root.findall(".//{%s}FUNDAMENTAL" %(ns)):
    if int(fundamental.get('MarketCap')) >= min_mcap:
        symbol_list.append([fundamental.get('Symbol'), fundamental.get('Name'), int(fundamental.get('MarketCap')),])

In [11]:
columns = ['yahoo_ticker', 'name', 'mcap']
df = pd.DataFrame(symbol_list, columns=columns)
df = df.sort_values(by='mcap', ascending=False).reset_index(drop=True)
df['yahoo_ticker'] = df['yahoo_ticker'] + '.AX'

symbol_list = None

In [12]:
stocks = df['yahoo_ticker'].values.tolist()
len(stocks)

583

In [13]:
%%time
prices = data.DataReader(stocks, 'yahoo', start, end)['Adj Close']

/home/adrian/miniconda3/envs/ds_ml_portfolio/lib/python3.5/site-packages/pandas_datareader/base.py:192: SymbolWarning:

Failed to read symbol: 'USP.AX', replacing with NaN.

/home/adrian/miniconda3/envs/ds_ml_portfolio/lib/python3.5/site-packages/pandas_datareader/base.py:192: SymbolWarning:

Failed to read symbol: 'JIP.AX', replacing with NaN.

/home/adrian/miniconda3/envs/ds_ml_portfolio/lib/python3.5/site-packages/pandas_datareader/base.py:192: SymbolWarning:

Failed to read symbol: 'AYH.AX', replacing with NaN.

/home/adrian/miniconda3/envs/ds_ml_portfolio/lib/python3.5/site-packages/pandas_datareader/base.py:192: SymbolWarning:

Failed to read symbol: 'ING.AX', replacing with NaN.

/home/adrian/miniconda3/envs/ds_ml_portfolio/lib/python3.5/site-packages/pandas_datareader/base.py:192: SymbolWarning:

Failed to read symbol: 'ATT.AX', replacing with NaN.

/home/adrian/miniconda3/envs/ds_ml_portfolio/lib/python3.5/site-packages/pandas_datareader/base.py:192: SymbolWarning:

Failed to 

CPU times: user 15 s, sys: 1.55 s, total: 16.6 s
Wall time: 9min 9s


In [14]:
stats = prices.describe().transpose()
cols_to_drop = list(stats[stats['count'] < min_days_traded].index)
len(cols_to_drop)

29

In [15]:
prices.drop(cols_to_drop, axis=1, inplace=True)
prices.iloc[:, 0:10].head()

,1AL.AX,3PL.AX,88E.AX,8IH.AX,A2M.AX,AAC.AX,AAD.AX,ABA.AX,ABC.AX,ABP.AX
Date,,,,,,,,,,
1999-01-04,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.75078,0.21648,NaN
1999-01-05,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.21380,NaN
1999-01-06,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.76647,0.20846,NaN
1999-01-07,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.78440,0.21380,NaN
1999-01-08,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.78440,0.21648,NaN


In [16]:
prices.fillna(method='ffill', inplace=True)
prices.iloc[:, 0:10].head()

,1AL.AX,3PL.AX,88E.AX,8IH.AX,A2M.AX,AAC.AX,AAD.AX,ABA.AX,ABC.AX,ABP.AX
Date,,,,,,,,,,
1999-01-04,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.75078,0.21648,NaN
1999-01-05,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.75078,0.21380,NaN
1999-01-06,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.76647,0.20846,NaN
1999-01-07,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.78440,0.21380,NaN
1999-01-08,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.78440,0.21648,NaN


In [19]:
iplot([{'x': prices.index, 'y': prices['CBA.AX']}])